<a href="https://colab.research.google.com/github/ForskyOnly/computer_vision/blob/main/Sign_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Project with Roboflow Datasets](https://ik.imagekit.io/roboflow/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://universe.roboflow.com/roboflow-100/sign-language-sokdr/model/1)
# Train YOLOv8 Object Detection on a Roboflow Dataset

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset)

Ultralytics YOLOv8 is the latest version of the YOLO (You Only Look Once) object detection and image segmentation model developed by Ultralytics. The YOLOv8 model is designed to be fast, accurate, and easy to use, making it an excellent choice for a wide range of object detection and image segmentation tasks. It can be trained on large datasets and is capable of running on a variety of hardware platforms, from CPUs to GPUs.



In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

## Install roboflow and ultralytics

In [ ]:
!pip install roboflow

In [ ]:
!pip install ultralytics

## Import data

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="8gLn8sffNZv2kFSo4IYP")
project = rf.workspace("roboflow-100").project("sign-language-sokdr")
dataset = project.version(2).download("yolov8")


In [ ]:
project.type

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
model = YOLO(f'/yolov8n-cls.pt')

## Training Model

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(data="/content/sign-language-2/data.yaml", epochs=15)

metrics = model.val()

path = model.export(format="onnx")

## Testing Model

In [ ]:
from ultralytics import YOLO
from PIL import Image
import cv2

model = model

#results = model.predict(source="folder", show=True) # Display preds. Accepts all YOLO predict arguments

#from PIL
im1 = Image.open("/content/sign-language-2/test/images/B14_jpg.rf.85f67b103a1aaef05506af61cb1ef72d.jpg")
results = model.predict(source=im1, save=True)  # save plotted images


In [ ]:
from IPython.display import display, Image

In [ ]:
Image(filename='/content/runs/detect/predict/B14_jpg.rf.85f67b103a1aaef05506af61cb1ef72d.jpg', height=300)

In [ ]:
from IPython.display import display, Image
Image(filename='/content/runs/detect/val/val_batch0_labels.jpg', height=600)

In [ ]:
from IPython.display import display, Image
Image(filename='/content/runs/detect/val/val_batch0_pred.jpg', height=600)

In [ ]:
from IPython.display import display, Image
Image(filename='/content/runs/detect/val/P_curve.png', height=500)

In [ ]:
import subprocess

# Définir les arguments de la commande YOLO
task = "detect"
mode = "predict"
model = "/content/runs/detect/train3/weights/best.pt"
conf = 0.25
source = f"/content/sign-language-2/test/images"
save = True

# Construire la commande YOLO
command = f"!yolo task={task} mode={mode} model={model} conf={conf} source={source} save={save}"

# Exécuter la commande shell
subprocess.call(command, shell=True)

In [ ]:
from ultralytics import YOLO
from PIL import Image

# Définir les chemins d'accès et les paramètres
model_path = "/content/runs/detect/train/weights/best.pt"
image_folder = "/content/sign-language-2/test/images"
conf_threshold = 0.25

# Charger le modèle YOLO
model = YOLO(model_path)

# Charger et prédire les images du dossier
results = model.predict(image_folder, conf=conf_threshold, save=True)

# Récupérer les chemins des images prédites
predicted_image_paths = results.save_dir.glob("*.jpg")

# Déplacer les images prédites vers le dossier souhaité
destination_folder = "/content/predict"
for image_path in predicted_image_paths:
    destination_path = destination_folder + "/" + image_path.name
    image_path.rename(destination_path)

# Afficher les chemins des images prédites
print("Chemins des images prédites :")
for image_path in predicted_image_paths:
    print(image_path)

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'/content/runs/detect/predict2/*.jpg')[:4]:
      display(Image(filename=image_path, width=200))
      print("\n")

# Transfer Learning


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

for param in model.model.parameters():
  param.requires_grad = False

model.train(data="/content/sign-language-2/data.yaml", epochs=15,  dropout=0.3, patience=5, batch=16, seed=42)

for param in model.model.parameters():
  param.requires_grad = True

model.train(data="/content/sign-language-2/data.yaml", epochs=5, patience=3, dropout=0.3, batch=16, seed=42)

metrics = model.val()

path = model.export(format="onnx")